<a href="https://colab.research.google.com/github/lorenzopaoria/Smoking-detection-and-distance-analysis/blob/main/person_cigarette_model_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run a model for sigarette and person detection

In [12]:
pip install ultralytics

In [13]:
from ultralytics import YOLO
import torch
from pathlib import Path
import cv2
import numpy as np
from tqdm.auto import tqdm
import os
import json

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class InferenceVisualizer:
    def __init__(self, model_path, conf_threshold=0.25):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model = YOLO(model_path)
        self.model.to(self.device)
        self.conf_threshold = conf_threshold
        self.image_counter = 1 

        self.colors = {
            0: (0, 255, 0),    # Green
            1: (255, 0, 0),    # Blue
            2: (0, 0, 255),    # Red
        }

    def save_detection_data(self, image_name, boxes, classes, confidences, output_dir, sequence_number):
        """Save detection data to a JSON file"""
        detections = []
        for box, cls, conf in zip(boxes, classes, confidences):
            detection = {
                'class': int(cls),
                'confidence': float(conf),
                'bbox': [float(x) for x in box],
            }
            detections.append(detection)

        data = {
            'image_name': image_name,
            'detections': detections
        }

        coords_dir = Path(output_dir) / 'coordinates'
        coords_dir.mkdir(parents=True, exist_ok=True)

        json_path = coords_dir / f"trained_{sequence_number}.json"
        with open(json_path, 'w') as f:
            json.dump(data, f, indent=2)

    def process_image(self, image_path, output_dir):
        """Process a single image and save both the annotated image and detection data"""
        image = cv2.imread(str(image_path))
        if image is None:
            print(f"Could not read image: {image_path}")
            return False

        results = self.model.predict(image, conf=self.conf_threshold)[0]
        annotated_image = image.copy()

        boxes = results.boxes.xyxy.cpu().numpy()
        classes = results.boxes.cls.cpu().numpy()
        confidences = results.boxes.conf.cpu().numpy()

        images_dir = Path(output_dir) / 'images'
        images_dir.mkdir(parents=True, exist_ok=True)

        for box, cls, conf in zip(boxes, classes, confidences):
            x1, y1, x2, y2 = box.astype(int)
            class_id = int(cls)
            color = self.colors.get(class_id, (0, 255, 0))

            cv2.rectangle(annotated_image, (x1, y1), (x2, y2), color, 2)
            label = f"Class {class_id}: {conf:.2f}"

            text_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
            text_x = x1
            text_y = y1 - 5 if y1 - 5 > text_size[1] else y1 + text_size[1]

            cv2.rectangle(annotated_image, (text_x, text_y - text_size[1] - 4), (text_x + text_size[0], text_y), color, -1)
            cv2.putText(annotated_image, label, (text_x, text_y - 2), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        output_image_path = images_dir / f"trained_{self.image_counter}.jpg"
        cv2.imwrite(str(output_image_path), annotated_image)

        self.save_detection_data(str(Path(image_path).name), boxes, classes, confidences, output_dir, self.image_counter)

        self.image_counter += 1  
        return True

    def process_directory(self, input_dir, output_dir):
        """Process all images in a directory with a reliable progress bar"""
        input_path = Path(input_dir)
        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)

        self.image_counter = 1

        image_files = []
        for ext in ['.jpg', '.jpeg', '.png']:
            image_files.extend(list(input_path.glob(f'*{ext}')))
            image_files.extend(list(input_path.glob(f'*{ext.upper()}')))

        total_images = len(image_files)
        print(f"Found {total_images} images")

        if total_images == 0:
            print("No images found in the input directory")
            return

        processed = 0
        failed = 0

        with tqdm(total=total_images, desc="Processing images", unit="img", ncols=80) as pbar:
            for img_path in image_files:
                try:
                    if self.process_image(img_path, output_path):
                        processed += 1
                    else:
                        failed += 1
                except Exception as e:
                    print(f"\nError processing {img_path.name}: {str(e)}")
                    failed += 1
                pbar.update(1)

        # Print summary
        print(f"\nProcessing complete!")
        print(f"Successfully processed: {processed} images")
        if failed > 0:
            print(f"Failed to process: {failed} images")
        print(f"Results saved in: {output_dir}")

In [16]:
def main():
    model_path = '/content/drive/MyDrive/pt_model_trained/run_20250211_142722/train/weights/best.pt'
    input_dir = '/content/drive/MyDrive/Photo_dataset/test/images'
    output_dir = '/content/drive/MyDrive/trained_photos'

    visualizer = InferenceVisualizer(model_path, conf_threshold=0.4)

    visualizer.process_directory(input_dir, output_dir)

    print("Elaborazione completata!")
    print(f"Le immagini elaborate sono state salvate in: {output_dir}")

if __name__ == "__main__":
    main()

Found 73 images


Processing images:   0%|                                | 0/73 [00:00<?, ?img/s]


0: 576x1024 2 cigarettes, 1 nonSmoker, 2 smokers, 13.8ms
Speed: 5.2ms preprocess, 13.8ms inference, 0.5ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 2 cigarettes, 1 nonSmoker, 3 smokers, 22.5ms
Speed: 7.7ms preprocess, 22.5ms inference, 0.7ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 2 cigarettes, 1 nonSmoker, 3 smokers, 16.8ms
Speed: 9.4ms preprocess, 16.8ms inference, 1.8ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 1 cigarette, 1 nonSmoker, 32.8ms
Speed: 8.3ms preprocess, 32.8ms inference, 0.9ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 2 cigarettes, 1 nonSmoker, 3 smokers, 15.0ms
Speed: 7.1ms preprocess, 15.0ms inference, 0.6ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 1 cigarette, 1 nonSmoker, 2 smokers, 14.8ms
Speed: 5.3ms preprocess, 14.8ms inference, 0.6ms postprocess per image at shape (1, 3, 576, 1024)

0: 576x1024 1 cigarette, 6 nonSmokers, 1 smoker, 30.3ms
Speed: 12.5ms prepro